## import stuff

In [14]:
#importing things
import pandas as pd
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import sem
from scipy.stats import ttest_ind_from_stats
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from collections import OrderedDict
import seaborn as sns

## fetch the data

In [25]:
#h1
h1a = pd.DataFrame(pd.read_csv('Kd_inference_9114_a_nova3.csv',delimiter=',',dtype={'sequence': str}))
h1b = pd.DataFrame(pd.read_csv('Kd_inference_9114_b_nova3.csv',delimiter=',',dtype={'sequence': str}))
h1c = pd.DataFrame(pd.read_csv('Kd_inference_9114_c_nova3.csv',delimiter=',',dtype={'sequence': str}))
#h3
h3d = pd.DataFrame(pd.read_csv('Kd_inference_9114_d_nova3.csv',delimiter=',',dtype={'sequence': str}))
h3e = pd.DataFrame(pd.read_csv('Kd_inference_9114_e_nova3.csv',delimiter=',',dtype={'sequence': str}))
h3f = pd.DataFrame(pd.read_csv('Kd_inference_9114_f_nova3.csv',delimiter=',',dtype={'sequence': str}))
#fluB
fluBg = pd.DataFrame(pd.read_csv('Kd_inference_9114_g_nova3.csv',delimiter=',',dtype={'sequence': str}))
fluBh = pd.DataFrame(pd.read_csv('Kd_inference_9114_h_nova3.csv',delimiter=',',dtype={'sequence': str}))
fluBi = pd.DataFrame(pd.read_csv('Kd_inference_9114_i_nova3.csv',delimiter=',',dtype={'sequence': str}))

## merging unfiltered data

In [26]:
#merge by antigen
#H1
h1_all = pd.DataFrame()
h1_all = pd.merge(h1a, h1b, on='sequence',how='outer')
h1_all = pd.merge(h1_all, h1c, on='sequence',how='outer')
#H3
h3_all = pd.DataFrame()
h3_all = pd.merge(h3d, h3e, on='sequence',how='outer')
h3_all = pd.merge(h3_all, h3f, on='sequence',how='outer')
#fluB
fluB_all = pd.DataFrame()
fluB_all = pd.merge(fluBg, fluBh, on='sequence',how='outer')
fluB_all = pd.merge(fluB_all, fluBi, on='sequence',how='outer')
print(h1_all.shape[0],h3_all.shape[0],fluB_all.shape[0])

#cleanup & calculate mean and SEM
#H1
h1_s = h1_all[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
h1_s['mean'] = h1_s.mean(axis=1)
h1_s['sem'] = h1_s.sem(axis=1)
h1_s.rename(columns={"log10_Kd_x": "repa", 'log10_Kd_y': "repb", 'log10_Kd': "repc","mean": "h1_mean", "sem": "h1_sem"},inplace=True)
#H3
h3_s = h3_all[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
h3_s['mean'] = h3_s.mean(axis=1)
h3_s['sem'] = h3_s.sem(axis=1)
h3_s.rename(columns={"log10_Kd_x": "repa", 'log10_Kd_y': "repb", 'log10_Kd': "repc","mean": "h3_mean", "sem": "h3_sem"},inplace=True)
#FluB
fluB_s = fluB_all[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
fluB_s['mean'] = fluB_s.mean(axis=1)
fluB_s['sem'] = fluB_s.sem(axis=1)
fluB_s.rename(columns={"log10_Kd_x": "repa", 'log10_Kd_y': "repb", 'log10_Kd': "repc","mean": "fluB_mean", "sem": "fluB_sem"},inplace=True)

65536 65536 65536


## Pinning Kd to boundaries, removing poor fits, merging replicates

In [28]:
#replacing things below the boundary with the boundary & then filtering out all poor fits
h1a_filt = h1a.copy()
column_name = "log10_Kd"
new_value = -7
boolean_condition = (h1a_filt.log10_Kd > -7)
h1a_filt.loc[boolean_condition, column_name] = new_value
print(h1a['log10_Kd'].mean(),h1a_filt['log10_Kd'].mean())
h1a_filt_out = h1a_filt.loc[((h1a_filt['log10_Kd'] < -7) & (h1a_filt['ste_log10Kd'] <= 1))|(h1a_filt['log10_Kd'] >= -7)]
h1a_filt_out.shape[0]

-8.79998480375719 -8.88299109355528


63608

In [29]:
#replacing things below the boundary with the boundary
h1b_filt = h1b.copy()
column_name = "log10_Kd"
new_value = -7
boolean_condition = (h1b_filt.log10_Kd > -7)
h1b_filt.loc[boolean_condition, column_name] = new_value
print(h1b['log10_Kd'].mean(),h1b_filt['log10_Kd'].mean())
h1b_filt_out = h1b_filt.loc[((h1b_filt['log10_Kd'] < -7) & (h1b_filt['ste_log10Kd'] <= 1))|(h1b_filt['log10_Kd'] >= -7)]
h1b_filt_out.shape[0]

-8.719029316111714 -8.886458810086003


63831

In [31]:
#replacing things below the boundary with the boundary
h1c_filt = h1c.copy()
column_name = "log10_Kd"
new_value = -7
boolean_condition = (h1c_filt.log10_Kd > -7)
h1c_filt.loc[boolean_condition, column_name] = new_value
print(h1c['log10_Kd'].mean(),h1c_filt['log10_Kd'].mean())
h1c_filt_out = h1c_filt.loc[((h1c_filt['log10_Kd'] < -7) & (h1c_filt['ste_log10Kd'] <= 1))|(h1c_filt['log10_Kd'] >= -7)]
h1c_filt_out.shape[0]

-8.682367814449728 -8.833032304109741


63797

In [32]:
#replacing things below the boundary with the boundary
h3d_filt = h3d.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (h3d_filt.log10_Kd > -6)
h3d_filt.loc[boolean_condition, column_name] = new_value
print(h3d['log10_Kd'].mean(),h3d_filt['log10_Kd'].mean())
h3d_filt_out = h3d_filt.loc[((h3d_filt['log10_Kd'] < -6) & (h3d_filt['ste_log10Kd'] <= 1))|(h3d_filt['log10_Kd'] >= -6)]
h3d_filt_out.shape[0]

-2.8034274538583266 -6.169967764112354


58645

In [33]:
#replacing things below the boundary with the boundary
h3e_filt = h3e.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (h3e_filt.log10_Kd > -6)
h3e_filt.loc[boolean_condition, column_name] = new_value
print(h3e['log10_Kd'].mean(),h3e_filt['log10_Kd'].mean())
h3e_filt_out = h3e_filt.loc[((h3e_filt['log10_Kd'] < -6) & (h3e_filt['ste_log10Kd'] <= 1))|(h3e_filt['log10_Kd'] >= -6)]
h3e_filt_out.shape[0]

-3.283516545673938 -6.277910939460245


52306

In [48]:
#replacing things below the boundary with the boundary
h3f_filt = h3f.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (h3f_filt.log10_Kd > -6)
h3f_filt.loc[boolean_condition, column_name] = new_value
print(h3f['log10_Kd'].mean(),h3f_filt['log10_Kd'].mean())
h3f_filt_out = h3f_filt.loc[((h3f_filt['log10_Kd'] < -6) & (h3f_filt['ste_log10Kd'] <= 1))|(h3f_filt['log10_Kd'] >= -6)]
h3f_filt_out.shape[0]

-2.3826395572428547 -6.126541625761835


60502

In [49]:
#replacing things below the boundary with the boundary
fluBg_filt = fluBg.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (fluBg_filt.log10_Kd > -6)
fluBg_filt.loc[boolean_condition, column_name] = new_value
print(fluBg['log10_Kd'].mean(),fluBg_filt['log10_Kd'].mean())
fluBg_filt_out = fluBg_filt.loc[((fluBg_filt['log10_Kd'] < -6) & (fluBg_filt['ste_log10Kd'] <= 1))|(fluBg_filt['log10_Kd'] >= -6)]
fluBg_filt_out.shape[0]

-2.8699333705702945 -6.224335623519193


55061

In [50]:
#replacing things below the boundary with the boundary
fluBh_filt = fluBh.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (fluBh_filt.log10_Kd > -6)
fluBh_filt.loc[boolean_condition, column_name] = new_value
print(fluBh['log10_Kd'].mean(),fluBh_filt['log10_Kd'].mean())
fluBh_filt_out = fluBh_filt.loc[((fluBh_filt['log10_Kd'] < -6) & (fluBh_filt['ste_log10Kd'] <= 1))|(fluBh_filt['log10_Kd'] >= -6)]
fluBh_filt_out.shape[0]

-3.0293431359043463 -6.289495083193853


54200

In [51]:
#replacing things below the boundary with the boundary
fluBi_filt = fluBi.copy()
column_name = "log10_Kd"
new_value = -6
boolean_condition = (fluBi_filt.log10_Kd > -6)
fluBi_filt.loc[boolean_condition, column_name] = new_value
print(fluBi['log10_Kd'].mean(),fluBi_filt['log10_Kd'].mean())
fluBi_filt_out = fluBi_filt.loc[((fluBi_filt['log10_Kd'] < -6) & (fluBi_filt['ste_log10Kd'] <= 1))|(fluBi_filt['log10_Kd'] >= -6)]
fluBi_filt_out.shape[0]

-3.0543139814647406 -6.279643685870487


54521

In [40]:
#merging
h1_filt_merg = pd.DataFrame()
h1_filt_merg = pd.merge(h1a_filt_out, h1b_filt_out, on='sequence',how='outer')
h1_filt_merg = pd.merge(h1_filt_merg, h1c_filt_out, on='sequence',how='outer')
h1_filt_merg_sm = h1_filt_merg[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
h1_filt_merg_sm['mean'] = h1_filt_merg_sm.mean(axis=1)
h1_filt_merg_sm['sem'] = h1_filt_merg_sm.sem(axis=1)
h1_filt_merg_sm.rename(columns={"log10_Kd_x": "repa", "log10_Kd_y": "repb", "log10_Kd": "repc","mean": "h1_mean", "sem": "h1_sem"},inplace=True)
h1_filt_merg_sm

,sequence,repa,repb,repc,h1_mean,h1_sem
0,0000000000000000,-7.898447,-8.142321,-8.454509,-8.165092,0.113791
1,0000000000000001,-8.050910,-7.936590,-8.464933,-8.150811,0.113484
2,0000000000000010,-8.384770,-8.026676,-8.371892,-8.261113,0.082928
3,0000000000000011,-8.563147,-7.770484,-8.112529,-8.148720,0.162307
4,0000000000000100,-8.385538,-7.977133,-8.568714,-8.310462,0.123639
...,...,...,...,...,...,...
65169,1110110100010001,NaN,NaN,-7.000000,-7.000000,0.000000
65170,1110111000011100,NaN,NaN,-7.000000,-7.000000,0.000000
65171,1110111000100100,NaN,NaN,-7.000000,-7.000000,0.000000
65172,1110111000100110,NaN,NaN,-7.000000,-7.000000,0.000000


In [41]:
#merging
h3_filt_merg = pd.DataFrame()
h3_filt_merg = pd.merge(h3d_filt_out, h3e_filt_out, on='sequence',how='outer')
h3_filt_merg = pd.merge(h3_filt_merg, h3f_filt_out, on='sequence',how='outer')
h3_filt_merg_sm = h3_filt_merg[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
h3_filt_merg_sm['mean'] = h3_filt_merg_sm.mean(axis=1)
h3_filt_merg_sm['sem'] = h3_filt_merg_sm.sem(axis=1)
h3_filt_merg_sm.rename(columns={"log10_Kd_x": "repa", "log10_Kd_y": "repb", "log10_Kd": "repc","mean": "h3_mean", "sem": "h3_sem"},inplace=True)
h3_filt_merg_sm

,sequence,repa,repb,repc,h3_mean,h3_sem
0,0000000000000000,-6.0,NaN,-6.0,-6.0,0.0
1,0000000000000001,-6.0,NaN,NaN,-6.0,0.0
2,0000000000000100,-6.0,-6.0,-6.0,-6.0,0.0
3,0000000000000101,-6.0,NaN,-6.0,-6.0,0.0
4,0000000000000110,-6.0,-6.0,-6.0,-6.0,0.0
...,...,...,...,...,...,...
65116,1111111000011111,NaN,NaN,-6.0,-6.0,0.0
65117,1111111010100000,NaN,NaN,-6.0,-6.0,0.0
65118,1111111100110001,NaN,NaN,-6.0,-6.0,0.0
65119,1111111101011000,NaN,NaN,-6.0,-6.0,0.0


In [42]:
#merging
fluB_filt_merg = pd.DataFrame()
fluB_filt_merg = pd.merge(fluBg_filt_out, fluBh_filt_out, on='sequence',how='outer')
fluB_filt_merg = pd.merge(fluB_filt_merg, fluBi_filt_out, on='sequence',how='outer')
fluB_filt_merg_sm = fluB_filt_merg[['sequence','log10_Kd_x','log10_Kd_y','log10_Kd']].copy()
fluB_filt_merg_sm['mean'] = fluB_filt_merg_sm.mean(axis=1)
fluB_filt_merg_sm['sem'] = fluB_filt_merg_sm.sem(axis=1)
fluB_filt_merg_sm.rename(columns={"log10_Kd_x": "repa", "log10_Kd_y": "repb", "log10_Kd": "repc","mean": "fluB_mean", "sem": "fluB_sem"},inplace=True)
fluB_filt_merg_sm

,sequence,repa,repb,repc,fluB_mean,fluB_sem
0,0000000000000000,-6.0,-6.0,-6.00000,-6.00000,0.0
1,0000000000000001,-6.0,-6.0,NaN,-6.00000,0.0
2,0000000000000010,-6.0,NaN,-6.00000,-6.00000,0.0
3,0000000000000011,-6.0,NaN,-6.00000,-6.00000,0.0
4,0000000000000100,-6.0,-6.0,-6.00000,-6.00000,0.0
...,...,...,...,...,...,...
64922,1111111001110010,NaN,NaN,-6.00000,-6.00000,0.0
64923,1111111010101110,NaN,NaN,-6.00000,-6.00000,0.0
64924,1111111010110010,NaN,NaN,-6.00000,-6.00000,0.0
64925,1111111110000110,NaN,NaN,-6.00000,-6.00000,0.0


## merging antigens: unfiltered, unadjusted data

In [45]:
#merge all antigens
HA_all_unf = pd.DataFrame()
HA_all_unf = pd.merge(h1_s, h3_s, on='sequence',how='outer')
HA_all_unf = pd.merge(HA_all_unf, fluB_s, on='sequence',how='outer')
HA_all_unf.to_csv('kd_processed/20210427_HA_unadj_unfilt_merg_ML.csv', index=False)

## merging antigens: filtered, only adjusted if outside titration boundaries

In [47]:
#merge all antigens
HA_all_fil = pd.DataFrame()
HA_all_fil = pd.merge(h1_filt_merg_sm, h3_filt_merg_sm, on='sequence',how='outer')
HA_all_fil = pd.merge(HA_all_fil, fluB_filt_merg_sm, on='sequence',how='outer')
HA_all_fil.to_csv('kd_processed/20210427_HA_unadj_fil_merg_ML.csv', index=False)